Begin by importing all necessary libraries.

In [1]:
import math
import pandas as pd
import numpy as np
%matplotlib inline

Then import all data files.

In [2]:
demo_ref = pd.read_csv('demographicref.csv') #demographic reference data
econ_ref = pd.read_csv('econref.csv') #economic reference data
health_expen = pd.read_csv('healthexpenditure.csv') #healthcare expenditure data
health_qual = pd.read_csv('healthquality.csv') #healthcare quality data
health_resor = pd.read_csv('healthresources.csv') #healthcare resources data
health_util = pd.read_csv('healthutil.csv') #healthcare utilization data
social_proc = pd.read_csv('socialprotection.csv') #social healthcare protection data
worker_migr = pd.read_csv('healthworkmigration.csv') #healthcare worker migration data

The final dataset cannot be loaded past the 284,614th line, so in order to avoid having partial data on one variable we needed to cut the last 40,000 or so entries.

In [3]:
health_stat = pd.read_csv('healthstatus.csv',nrows = 284583) #health status data

Remove unwanted columns of data.

In [4]:
demo_ref = demo_ref[['Variable','Measure','Country','Year','Value']]
econ_ref = econ_ref[['Variable', 'Measure', 'Country','Year', 'Value']]
health_expen = health_expen[['Financing scheme', 'Function','Provider', 'Measure', 'Country', 'Year', 'Unit', 'Value']]
health_qual = health_qual[['Country','Periods','Indicator','Gender','Age Group','Value','Value.1']]
health_resor = health_resor[['Variable','Measure','Country','Year','Value']]
health_util = health_util[['Variable','Measure','Country','Year','Value']]
social_proc = social_proc[['Variable','Measure','Country','Year','Value']]
worker_migr = worker_migr[['Country','Variable','Country of origin','Year','Value']]
health_stat = health_stat[['Variable','Measure','Country','Year','Value']]

Remove unwanted rows of data. Typically rows containing data measured in units that cannot be compared between countries (e.g. in local currency or raw numbers of incidents). Also removing rows containing variables with few observations or which are unlikely to be helpful for our purposes. Not all data sets required this.

In [43]:
econ_ref = econ_ref.loc[(econ_ref['Measure']=='Price index (2015=100)')|
                       (econ_ref['Measure']=='/capita, US$ purchasing power parity')|
                       (econ_ref['Measure']=='Current prices in NCU')]

health_expen = health_expen.loc[health_expen['Measure']=='Share of gross domestic product']

health_qual = health_qual.loc[((health_qual['Value']=='Age-sex standardised rate per 100 000 population')|
               (health_qual['Value']=='Age-sex standardised rate per 100 patients')|
               (health_qual['Value']=='Age-standardised survival (%) '))&
                (health_qual['Gender']=='Total')] 

health_resor = health_resor.loc[(health_resor['Measure']=='Density per 1 000 population (head counts)')|
                (health_resor['Measure']=='Per million population')|
                (health_resor['Measure']=='% of total physicians (head counts)')|
                (health_resor['Measure']=='Per 1 000 population')|
                (health_resor['Measure']=='% of physicians (head counts)')|
                (health_resor['Measure']=='Per 100 000 population')|
                (health_resor['Measure']=='% of total hospital employment (head counts)')|
                (health_resor['Measure']=='Per 1 000 live births')|
                (health_resor['Measure']=='Salaried, income, US$ exchange rate')] 

health_util = health_util.loc[(health_util['Measure']=='Per 100 000 population')|
               (health_util['Measure']=='% performed as inpatient cases')|
               (health_util['Measure']=='% performed as day cases')|
               (health_util['Measure']=='Per 100 000 females')|
               (health_util['Measure']=='Inpatient cases per 100 000 population ')|
               (health_util['Measure']=='Total procedures per 100 000 population')|
               (health_util['Measure']=='Day cases per 100 000 population')|
               (health_util['Measure']=='Per 1 000 population')]

social_proc = social_proc.loc[(social_proc['Measure']=='% of total population')|(social_proc['Measure']=='% of total population covered')] 

health_stat = health_stat.loc[(health_stat['Measure']=='Deaths per 100 000 females (standardised rates)')|
               (health_stat['Measure']=='Deaths per 100 000 females (crude rates)')|
               (health_stat['Measure']=='Years lost, /100 000 females, aged 75 years old')|
               (health_stat['Measure']=='Deaths per 100 000 males (standardised rates)')|
               (health_stat['Measure']=='Deaths per 100 000 population (standardised rates)')|
               (health_stat['Measure']=='Deaths per 100 000 males (crude rates)')|
               (health_stat['Measure']=='Years lost, /100 000 males, aged 75 years old')|
               (health_stat['Measure']=='Years lost, /100 000 population, aged 75 years old')|
               (health_stat['Measure']=='Deaths per 100 000 population (crude rates)')|
               (health_stat['Measure']=='% of population (crude rate)')|
               (health_stat['Measure']=='Years')]

Next we have to clean and tidy the data.

In [62]:
demo_ref_pv = demo_ref.pivot_table(index = ['Country','Year'], columns = ['Variable','Measure'], values = 'Value')
econ_ref_pv = econ_ref.pivot_table(index = ['Country','Year'], columns = ['Variable','Measure'], values = 'Value')
health_expen_pv = health_expen.pivot_table(index = ['Country','Year'], columns = ['Function','Measure'], values = 'Value')
health_qual_pv = health_qual.pivot_table(index = ['Country','Periods','Age Group'], columns = ['Indicator','Value'], values = 'Value.1')
health_resor_pv = health_resor.pivot_table(index = ['Country','Year'], columns = ['Variable','Measure'], values = 'Value')
health_util_pv = health_util.pivot_table(index = ['Country','Year'], columns = ['Variable','Measure'], values = 'Value')
social_proc_pv = social_proc.pivot_table(index = ['Country','Year'], columns = ['Variable','Measure'], values = 'Value')
health_stat_pv = health_stat.pivot_table(index = ['Country','Year'], columns = ['Variable','Measure'], values = 'Value')

Variable                                   Accidental falls  \
Measure            Deaths per 100 000 females (crude rates)   
Country       Year                                            
Australia     2010                                      7.9   
              2011                                      8.6   
              2012                                      9.4   
              2013                                      8.9   
              2014                                     10.7   
...                                                     ...   
United States 2017                                     10.9   
              2018                                     11.2   
              2019                                     11.6   
              2020                                     12.4   
              2021                                      NaN   

Variable                                                            \
Measure            Deaths per 100 000 females (standardised rates)   
Country       Year                                                   
Australia     2010                                             7.4   
              2011                                             8.0   
              2012                                             8.6   
              2013                                             8.2   
              2014                                             9.7   
...                                                            ...   
United States 2017                                             9.8   
              2018                                            10.0   
              2019                                            10.3   
              2020                                            10.8   
              2021                                             NaN   

Variable                                                   \
Measure            Deaths per 100 000 males (crude rates)   
Country       Year                                          
Australia     2010                                    7.3   
              2011                                    8.2   
              2012                                    8.4   
              2013                                    8.3   
              2014                                    9.7   
...                                                   ...   
United States 2017                                   11.5   
              2018                                   11.7   
              2019                                   12.4   
              2020                                   13.1   
              2021                                    NaN   

Variable                                                          \
Measure            Deaths per 100 000 males (standardised rates)   
Country       Year                                                 
Australia     2010                                          10.1   
              2011                                          11.4   
              2012                                          11.4   
              2013                                          11.1   
              2014                                          12.9   
...                                                          ...   
United States 2017                                          14.3   
              2018                                          14.4   
              2019                                          15.0   
              2020                                          15.7   
              2021                                           NaN   

Variable                                                        \
Measure            Deaths per 100 000 population (crude rates)   
Country       Year                                               
Australia     2010                                         7.6   
              2011                                         8.4  